<a href="https://colab.research.google.com/github/ShotaSasaki-HCU/Prog2kakushin/blob/main/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
# Pandas
# output.clear() # 出力画面をクリアする。

import pandas as pd
from google.colab import output # 出力画面クリア用

df = pd.read_csv('https://raw.githubusercontent.com/ShotaSasaki-HCU/Prog2kakushin/main/netflix%20daily%20top%2010.csv')

separate = '-----------------------------------------------------------' # 区切り横線
print("【2020年4月1日〜2022年3月11日のNetflixランキング分析】")
while True:
  print("1. 最も多くトップ10に現れた作品")
  select = int(input(separate + "\n選択肢の番号を入力："))

  # 1. 最も多くトップ10に現れた作品
  if select == 1:
    output.clear() # 出力画面をクリアする。
    print("【1. 最も多くトップ10に現れた作品】")
    df_title = df['Title'].value_counts() # ユニークな要素のそれぞれの出現回数を得る。
    while True:
      limit = int(input("上位何位(1~" + str(len(df_title)) + ")まで表示しますか？："))
      if 1 <= limit <= len(df_title): # 入力がデータの範囲内ならば。
        break
      else: # 入力がデータの範囲外ならば。
        print("[エラー]1~" + str(len(df_title)) + "の範囲で入力してください。")
    titles = df_title[0:limit].index.values # 出現回数が格納されたSeriesの行名（作品タイトル）をリスト化。
    print('Title'.center(33) + ':\t出現日数（／' + str(int(len(df)/10)) + '日）\n' + separate)
    for i in range(len(titles)):
      print(str(i+1) + '. ' + titles[i].center(30) + ':\t' + str(df_title[0:limit][i]).center(16))

  break

【1. 最も多くトップ10に現れた作品】
上位何位(1~645)まで表示しますか？：10
              Title              :	出現日数（／710日）
-----------------------------------------------------------
1.           Cocomelon           :	      428       
2.             Ozark             :	       85       
3.           Cobra Kai           :	       81       
4.            Manifest           :	       80       
5.      The Queenâs Gambit     :	       73       
6.          Outer Banks          :	       72       
7.           Squid Game          :	       66       
8.          All American         :	       58       
9.           Bridgerton          :	       58       
10.            Lucifer            :	       56       
